In [1]:
import re
from typing import Dict, List

from pyswip import Prolog

In [2]:
prolog = Prolog()
prolog.assertz("father(michael,john)")
prolog.assertz("father(michael,gina)")
list(prolog.query("father(michael,X)"))  # [{'X': 'john'}, {'X': 'gina'}]

[{'X': 'john'}, {'X': 'gina'}]

In [3]:
cryptarithms = [
    "SEND + MORE == MONEY",
    "VIOLIN + VIOLA == TRIO",
    "ODD + ODD == EVEN",
]

In [4]:
def get_vars(expr: str) -> List[str]:
    return list(set(re.findall(r"[A-Z]", expr)))

In [5]:
get_vars(cryptarithms[0])

['R', 'M', 'D', 'N', 'Y', 'O', 'S', 'E']

In [6]:
rules = [
    "digit(X) :- member(X, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])",
    "all_diff([])",
    "all_diff([H|T]) :- \+member(H, T), all_diff(T)",
    """
    solve([O, D, D], [O, D, D], [E, V, E, N]) :-
        % génération des chiffres
        digit(O), digit(D),
        digit(E), digit(V), digit(N),
        % test que O et E sont différents de 0
        O =\= 0, E =\= 0,
        % test de la somme
        all_diff([O, D, E, V, N]),
                   100 * O + 10 * D + D +
                   100 * O + 10 * D + D =:=
        1000 * E + 100 * V + 10 * E + N
    """,
]
for rule in rules:
    prolog.assertz(rule)

In [7]:
query = "solve([O, D, D], [O, D, D], [E, V, E, N])"
list(prolog.query(query))

[{'O': 6, 'D': 5, 'E': 1, 'V': 3, 'N': 0},
 {'O': 8, 'D': 5, 'E': 1, 'V': 7, 'N': 0}]

In [20]:
def all_digit(expr: str) -> List[str]:
    return [f"digit({char})" for char in get_vars(expr)]

In [21]:
all_digit(cryptarithms[2])

['digit(V)', 'digit(D)', 'digit(N)', 'digit(O)', 'digit(E)']

In [16]:
def all_diff(expr: str) -> str:
    return f"all_diff([{', '.join(get_vars(expr))}])"

In [17]:
all_diff(cryptarithms[2])

'all_diff([V, D, N, O, E])'

In [34]:
def generate(
    expr: str, allow_zero: bool = True, allow_leading_zero: bool = False
) -> List[str]:
    generated = []
    generated += all_digit(expr)
    generated.append(all_diff(expr))
    if not allow_zero:
        for char in get_vars(expr):
            generated.append(f"dif({char}, 0)")
    if not allow_leading_zero:
        for char in set(re.findall(r'\b(\w)', expr)):
            generated.append(f"dif({char}, 0)")
    return generated

In [35]:
generate(cryptarithms[2])

['digit(V)',
 'digit(D)',
 'digit(N)',
 'digit(O)',
 'digit(E)',
 'all_diff([V, D, N, O, E])',
 'dif(V, 0)',
 'dif(D, 0)',
 'dif(N, 0)',
 'dif(O, 0)',
 'dif(E, 0)',
 'dif(O, 0)',
 'dif(E, 0)']